# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [13]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
# Bayesian Parameter Sampling
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice




## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [14]:
ws = Workspace.from_config()
experiment_name = 'customized-rain'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: coursera_paes
Azure region: brazilsouth
Subscription id: 90b94744-e13a-4ecb-a955-1cbb85261a5d
Resource group: defaultresourcegroup-brazilsouth


# Create a Cluster

In [19]:

#cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "vzeizer1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster=ComputeTarget(workspace=ws,name=cluster_name)
    print('Cluster already exists!')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
    cpu_cluster = ComputeTarget(ws,cluster_name,compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Cluster already exists!

Running


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [38]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# I am going to put Bayesian termination policy
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.15, 
                                        slack_amount=None, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
# this is easy, from LR,C strength and max iter
param_sampling = BayesianParameterSampling({
    '--C':choice(0.01,0.1,1.0,10.,100.),
    "--max_iter":choice(100,500,1000)
})
env=Environment.from_conda_specification(name="myenv",
                                                  file_path="./env.yml")
project_folder='./project_files'
os.makedirs(project_folder,exist_ok=True)

#TODO: Create your estimator and hyperdrive config
#estimator = SKLearn(source_directory=project_folder, 
#                    script_params=param_sampling,
#                    compute_target=cpu_cluster,
#                    entry_script='train_tomorrow.py',
#                   )
# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train_raintomorrow.py',
                            compute_target=cpu_cluster,
                            environment=env
                            )### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
#                    estimator=estimator,
                    hyperparameter_sampling=param_sampling,
                    policy=early_termination_policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=10,
                    max_concurrent_runs=4)### YOUR CODE HERE ###



For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:40.


In [39]:
#TODO: Submit your experiment
run_hyperdrive=experiment.submit(config=hyperdrive_config)
run_hyperdrive.wait_for_completion(show_output=True)


RunId: HD_5745ca5c-b268-4522-b1f5-7bf52372762c
Web View: https://ml.azure.com/runs/HD_5745ca5c-b268-4522-b1f5-7bf52372762c?wsid=/subscriptions/90b94744-e13a-4ecb-a955-1cbb85261a5d/resourcegroups/defaultresourcegroup-brazilsouth/workspaces/coursera_paes&tid=51d21047-48d2-44d8-9b11-c3c090deec69

Streaming azureml-logs/hyperdrive.txt

[2023-03-04T15:14:09.057787][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-03-04T15:14:09.6522434Z][SCHEDULER][INFO]Scheduling job, id='HD_5745ca5c-b268-4522-b1f5-7bf52372762c_0' 
[2023-03-04T15:14:09.7986953Z][SCHEDULER][INFO]Scheduling job, id='HD_5745ca5c-b268-4522-b1f5-7bf52372762c_1' 
[2023-03-04T15:14:09.9796002Z][SCHEDULER][INFO]Scheduling job, id='HD_5745ca5c-b268-4522-b1f5-7bf52372762c_2' 
[2023-03-04T15:14:09.978614][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-03-04T15:14:10.1262323Z][SCHEDULER][INFO]Scheduling job, id='HD_5745ca5c-b268-4522-b1f5-7bf523727

{'runId': 'HD_5745ca5c-b268-4522-b1f5-7bf52372762c',
 'target': 'vzeizer1',
 'status': 'Completed',
 'startTimeUtc': '2023-03-04T15:14:08.570708Z',
 'endTimeUtc': '2023-03-04T15:20:22.915783Z',
 'services': {},
 'error': {'error': {'code': 'UserError',
   'message': "Execution failed. User process '/azureml-envs/azureml_e99c7c30cae8d63464ab733555b94621/bin/python' terminated by signal 15. Please check log file 'user_logs/std_log.txt' for error details. Error: train_raintomorrow.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`\n  df[encoded_cols]=encoder.transform(df[categorical_cols])\ntrain_raintomorrow.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [40]:
RunDetails(run_hyperdrive).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [41]:
best_run = run_hyperdrive.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Best run metrics:', best_run_metrics)
print('\n parameter values:', parameter_values)


Best Run Id:  HD_5745ca5c-b268-4522-b1f5-7bf52372762c_1

 Best run metrics: {'Regularization Strength:': 100.0, 'Max iterations:': 1000, 'Accuracy': 0.8319583848938998}

 parameter values: ['--C', '100', '--max_iter', '1000']


In [42]:
#TODO: Save the best model
saved_model=best_run.register_model(model_name='best-hyperdrive',model_path='./')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# need to register both customized and AutoML best models

#from here "https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-deploy-and-where?tabs=python"

#env = Environment.get(ws,'customized-rain')
#python_packages = ['scikit-learn']
#for package in python_packages:
# env -> estimator
#    env.python.conda_dependencies.add_pip_package(package)

inference_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./score.py')



# from: "https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-deploy-and-where?tabs=python"
# Register model
model = Model.register(ws, model_name="customized_model", model_path="./best-hyperdrive",
                      model_framework=)



service_name = 'bestmodel-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

from azureml.core.webservice import LocalWebservice
port='8000'
deployment_config = LocalWebservice.deploy_configuration(port=port)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:"+port)
headers = {"Content-Type": "application/json"}
data = {
    'MaxTemp':28.5, 'Rainfall':0.7, 'Sunshine':7.62, 'WindGustSpeed':77.0, 
    'Humidity9am':58.0, 'Humidity3pm':26.0, 'Pressure9am':1019.0, 'Pressure3pm':1016.0,
    'Cloud9am':6.8, 'Cloud3pm':5.7, 'Temp3pm':25, 'RainToday':1.0}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

# Delete the service

In [ ]:
print(service.get_logs())
service.delete()

# Delete model

In [ ]:
model.delete()

# Delete the cpu cluster

In [ ]:
cpu_cluster.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

